## Monte Carlo Geometry Processing

Monte Carlo PDE estimator computes the solution to a partial differential equation at some given point with the Walk on Spheres(WoS) algorithm.


### Laplacian Equation


For a Laplacian equation given by 
$$ \Delta u = 0 \ \ \ on \ \ \Omega$$
$$ u = g \ \ \ on \ \ \partial\Omega$$
The WoS algorithm is done iteratively, for an iteration k:
1. At $x_k$, find the largest sphere $S(x_k)$ centered at $x_k$ inscribed in the mesh
2. Pick a random point $x_{k+1}$ on $S(x_k)$

Repeat step 1 and 2 until $x_k$ is close to the mesh, i.e. $d(x_k, V, F) < \epsilon$  
Give an estimate for $u(x_0)$ as $\hat{u}(x_0)$ = $g(\bar{x_k})$, where $\bar{x_k}$ is the closest point on mesh to $x_k$

In libigl, this estimator can be used as follows:

Where $B$ is a function $\mathbb{R}^3 \rightarrow \mathbb{R}$ that can be evaluated on points on mesh. $P$ is a set of Query points (not necessarily part of any mesh).

<table><tr>
<td> <img src="images/bunny_laplacian.png" style="width: 500px;"/> </td>
<td> <img src="images/cactus_laplacian.png" style="width: 500px;"/> </td>
</tr></table>

Figure 1. Solved laplacian with the boundary condition $g(x) = \|x\|_2$ 

### Poisson Equation

For a Poisson equation given by
$$\Delta u = f \ \ \ on \ \ \Omega$$
$$u = g \ \ \ on\ \  \partial\Omega$$

The WoS algorithm is similar to the one for Laplacian, except for an iteration k, it has an extra step:
3. Pick a random point $y_k$ in the largest ball $B(x_k)$ centered at $x_k$ inscribed in the mesh

And the estimate for $u(x_0)$ is $\hat{u}(x_0) = g(\bar{x_k}) + \Sigma_{i=1}^k |B(x_i)|f(y_i)G(x_i, y_i)$

In libigl, this estimator can be used as follows:

Where $f:\mathbb{R}^3 \rightarrow \mathbb{R}$ is a source function that can be specified by the user

#### Importance Sampling

For a specific type of source function $f_z = c \delta_z$, where c is a constant and 
$\delta_z$ is the Dirac delta centered at a point $z\in \Omega$, we can instead sample $y_k$ in the following way. 

3*. Pick a random point $y_k$ in the largest ball $B(x_k)$ centered at $x_k$ inscribed in the mesh, and if $y_k$ is close enough to the source point $z$ ($|y_k - z| < r$), use the following term for the contribution

$$|B(x_k)|\frac{f(z)G(x,z)}{p(z)} = |B(x_k)|\frac{c \delta_z G(x, z)}{\delta_z} = |B(x_k)|c G(x, z)$$

Otherwise use the normal poisson contribution $|B(x_k)|f(y_k)G(x_k, y_k)$.

Importance sampling can be toggled with the double c and the use_importance boolean flag
TODO: paper section 4.2.2

<table><tr>
<td> <img src="images/with_importance.png" style="width: 500px;"/> </td>
<td> <img src="images/without_importance.png" style="width: 500px;"/> </td>
</tr></table>

Figure 2. Without importance sampling and 
with importance sampling using parameters c = 10000, z = (0.5, 0.5, 0.5) 

#### Screened Poisson Equation




A variant of the Poisson equation introduces a linear $u$ term
$$\begin{align} \Delta u - cu &= f \ \ \ on \ \ \Omega \\ u &= g \ \ \ on\ \  \partial\Omega \end{align}$$
which can be solved using similar methods to the original poisson equation except the contribution term is multiplied by a factor of 
$$\frac{R \sqrt{c}}{\sinh(R \sqrt{c}}$$

Similarly, importance sampling can be toggled with

### Biharmonic Equation

The biharmonic equation given by
$$\begin{align} \Delta^2 u &= 0 \ \ \ on \ \ \Omega \\ u &= g \ \ \ on\ \  \partial\Omega \\ \Delta u &= h \ \ \ on\ \  \partial\Omega \end{align}$$ 

The equation be simplified into a system of pdes using the substitution $v := \Delta u$ 

$$\begin{align} \Delta^2 u &=v \ \ \ on \ \ \Omega \\ \Delta v &= 0 \ \ \ on\ \  \partial\Omega \\ \Delta u &= g \ \ \ on\ \  \partial\Omega \\ \Delta v &= h \ \ \ on\ \  \partial\Omega \end{align}$$


The previous methods can be used to solve this equation, and 

The method is used as follows

where $B, h: \mathbb{R}^3 \rightarrow \mathbb{R}$ are boundary condition functions.